# Handy translator in your Python console using WordReference

If you have read any of my notebooks, you might have noticed that my English level is not outstanding. What if I wrote a Python script that, after introducing a Spanish word, goes to <a href="https://www.wordreference.com/">WordReference</a> and suggests some possible English translations for that word?

Well, thanks to <a href="https://www.wordreference.com/">WordReference</a> and its brilliantly-organized web architecture, this is possible. Welcome to my handy <a href="https://www.wordreference.com/">WordReference</a> translator from Spanish to English!

## Step 1: Load libraries

It is required to load the following libraries for running this program_

- **requests** and **BeautifulSoup**, as usual, for efficiently extracting the data from the Internet.
- **time**, to meet the requirements established by WebReference <a href="https://www.wordreference.com/robots.txt">robots.txt</a> criteria for crawlers. It is compulsary to wait 30 seconds between requests to WebReference when using a web-crawler. Although the objective of this script is just to make isolated consults, I have added one last line of code to fulfill this requirement just in case. 

In [1]:
import requests
import time
from bs4 import BeautifulSoup

## Step 2: Introduce the word you want to translate and see the results!

In the next block of code, after introducing an only Spanish word, the WordReference page containing its translations to English is requested (with `get` function in **requests** module) and parsed (with **BeautifulSoup**'s HTML parser). All translations are found in tables (between **table** tags, with class attribute being equal to **WRD**). In each table, all rows are extracted (within **tr** tags with class attribute being **odd** or **even**). Then, my code obtains all cells in each row (within **td** tags). 

Next, I loop over all cells in each row (`for all_td_in_row in all_td:`; and `for td in all_td_in_row`). Each cell is examined, and there are four possible situations:

- The cell contains <u>the word to be translated</u>: it meets the second condition in the if statement (class attribute is equal to **FrWrd**), so the word is printed to the console after "- If you want to translate ".
- The cell contains <u>the context in which the word translated would be used</u>: it meets the first condition in the if statement (class attribute is absent), so the context is printed to the console after "in the sense of ".
- The cell contains <u>the translated word</u>. it meets the third condition in the if statement (class attribute is **ToWrd**), so the translation is printed to the console after "then translate it to " (if it is the first translated word) or after "or ".
- The cell contains <u>no interesting information</u>: the cell is ignored and nothing is printed to the console.

Finally, my code waits 30 seconds before making another request to <a href="https://www.wordreference.com/">WordReference</a>.

In [2]:
url = "https://www.wordreference.com/es/en/translation.asp?spen={}"
page = requests.get(url.format(input("Introduce word: ")))
soup = BeautifulSoup(page.text, "html.parser")
tables = soup.findAll("table", class_="WRD")
  
mode = [0,1]
# This counter was introduced for a nice printing of the results
# The first position indicates the nature of the item to be printed
# The second position talks about the nature of the last printed item
# 0 indicates "FrWord" class
# 1 indicates "ToWrd" class

for table in tables:
    rows = table.findAll("tr", {"class": ["odd", "even"]})
    all_td = [row.findAll("td") for row in rows]
    for all_td_in_row in all_td:
        for td in all_td_in_row:
            try:
                if td.get("class") == None and td.text.strip() != "":
                    print("   in the sense of '", td.text.strip(), "',", sep = "")
                elif td.get("class")[0] == "FrWrd": 
                    mode[0] = 0
                    print("") if mode == [0,1] else True
                    word = list(td.children)[0].text.strip()
                    print("- If you want to translate '{}'".format(word) if \
                        mode[1] == 1 else "{}or '{}'".format(" "*24, word))
                    mode[1] = 0
                    
                elif td.get("class")[0] == "ToWrd":
                    mode[0] = 1
                    print("    then translate it to " if mode[1] == 0 else "{}or ".format(" "*22), end = "")
                    mode[1] = 1
                    translation = "'"
                    for child in td.children:
                        if child.name == None:
                            translation += (child.strip() + " ")
                        elif child.name == "span":
                            translation += (child.text.strip() + " ")
                    translation = translation.strip() + "'"
                    print(translation)
    
            except:
                continue
time.sleep(30)

Introduce word: traductor

- If you want to translate 'traductor, traductora'
   in the sense of '(persona que traduce) (multi-lingual person)',
    then translate it to 'translator'

- If you want to translate 'traductor, traductora'
   in the sense of '(programa informático) (translation program)',
    then translate it to 'automatic translator'

- If you want to translate 'nota del traductor'
   in the sense of '(puntualización del traductor)',
    then translate it to 'translator's note'

- If you want to translate 'traductor jurado, traductora jurada'
   in the sense of '(oficial, certificado)',
    then translate it to 'sworn translator'

- If you want to translate 'traductor literario, traductora literaria'
   in the sense of '(de textos, libros, filmes)',
    then translate it to 'literary translator'

- If you want to translate 'traductor público, traductora pública'
   in the sense of '(oficial, certificado)',
    then translate it to 'certified translator'


I hope you have enjoyed this script. Also, I hope this program will help me to write better in English in the future!